# Document cube generation...

Documentation on data cubes for DeepCube users.


## Prerequisites
Need a working python3 installation.

If not install Miniconda3 and from there python3.

Create environment with e.g. python3

Install the following packages: xarray, netCDF4, scipy, zarr, rasterio, dask

Need a working gdal installation to repoject data if needed.

In [ ]:
# activate conda environment
%conda activate myenv
# install python packages
%conda install xarray netCDF4 scipy zarr rasterio dask fsspec --upgrade

## Create scratch directory

In [ ]:
import os
myscratch = "/Net/Groups/BGI/scratch/mweynants"
if (not(os.path.isdir(myscratch))) :
    os.mkdir(myscratch)
# set myscratch as current directory
os.chdir(myscratch)
os.getcwd()

## Create cube with xarray {#minicube}
For **small cubes**, one can directly use `xarray` (https://xarray.pydata.org/en/stable/getting-started-guide/installing.html).

This is a minimal example on how to build data cubes using `xarray` only. It is assumed that all inputs are already transformed to a EPSG:4326 longitude-latitude grid. 

In [1]:
import xarray as xr
import dask
import numpy as np

era = xr.open_dataset("/Net/Groups/BGI/scratch/fgans/uc3-mini-dataset/ERA5-LAND/era5-land-hourly.nc")
clc = xr.open_dataset("/Net/Groups/BGI/scratch/fgans/uc3-mini-dataset/CLC-2018-CROPPED_NC/cropped_CLC_2018.nc")

The next step would be to decide for a target spatial resolution and to create regridded representations of all input datasets. Here we assume that the target would be the CLC resolution. Then we can use xarrays interp method to interpolate the data. The interpolation will fall back to scipy's interpolation methods and allows linearandnearest interpolations. For more complex/individual regridding, one can use gdal (see previous step) or xcube (next section).

In [2]:
clc

<xarray.Dataset>
Dimensions:  (lat: 2445, lon: 2957)
Coordinates:
  * lat      (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
  * lon      (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
Data variables:
    Band1    (lat, lon) float32 ...
    crs      |S1 ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.0.2, released 2019/10/28
    history:      Tue Mar 30 10:35:08 2021: GDAL Create( testout1.nc, ... )

In [23]:
clc.crs

<xarray.DataArray 'crs' ()>
array(b'', dtype='|S1')
Attributes:
    grid_mapping_name:            latitude_longitude
    long_name:                    CRS definition
    longitude_of_prime_meridian:  0.0
    semi_major_axis:              6378137.0
    inverse_flattening:           298.257223563
    spatial_ref:                  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...
    GeoTransform:                 20.82269567340527 0.0009946491976185691 0 3...

In [3]:
era

<xarray.Dataset>
Dimensions:    (longitude: 61, latitude: 91, time: 336)
Coordinates:
  * longitude  (longitude) float32 20.0 20.1 20.2 20.3 ... 25.7 25.8 25.9 26.0
  * latitude   (latitude) float32 39.0 38.9 38.8 38.7 ... 30.3 30.2 30.1 30.0
  * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    skt        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-05 13:05:41 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [4]:
target_lon = clc.lon
target_lat = clc.lat
# convert era into dask arrays with a single block and interpolate along lon and lat
era_interp = era.chunk().interp({'longitude':target_lon, 'latitude':target_lat},  method = 'linear')

/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devn

In [5]:
era_interp

<xarray.Dataset>
Dimensions:    (time: 336, lat: 2445, lon: 2957)
Coordinates:
  * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
    longitude  (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
    latitude   (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
  * lon        (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
  * lat        (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
Data variables:
    u10        (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    d2m        (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    skt        (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    sp         (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
    tp         (time, lat, lon) float32 dask.array<chunksize=(336, 2445, 2957), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-05 13:05:41 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

Please note that it is **essential to call chunk on the dataset to be regridded first to convert it to a dask array which is evaluated lazily**.

[xarray.DataArray.chunk](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.chunk.html)

[xarray.Dataset.to_zarr](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_zarr.html)

In the next step we would collect all new output variables into one dictionary

In [13]:
output_vars = {}
for k in era_interp.keys():
    output_vars[k] = era_interp[k]
output_vars['clc'] = clc['Band1']

In [14]:
output_vars

{'u10': <xarray.DataArray 'u10' (time: 336, lat: 2445, lon: 2957)>
 dask.array<transpose, shape=(336, 2445, 2957), dtype=float32, chunksize=(336, 2445, 2957), chunktype=numpy.ndarray>
 Coordinates:
   * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
     longitude  (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
     latitude   (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
   * lon        (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
   * lat        (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
 Attributes:
     units:      m s**-1
     long_name:  10 metre U wind component,
 'v10': <xarray.DataArray 'v10' (time: 336, lat: 2445, lon: 2957)>
 dask.array<transpose, shape=(336, 2445, 2957), dtype=float32, chunksize=(336, 2445, 2957), chunktype=numpy.ndarray>
 Coordinates:
   * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
     longitude  (lon) float64 20.82 20.82 20.83 

Then we create a new dataset from these DataArrays, select a target chunking and write the result to a new zarr file.

In [ ]:
ds = xr.Dataset(output_vars).chunk({'lon':200, 'lat':200}).to_zarr("./mydatacubelinear.zarr")
ds

Note that this new dataset has identical spatial resolutions and through the small spatial chunks will have fast access to individual time series.

In [2]:
import zarr
ds = xr.open_dataset("/Net/Groups/BGI/scratch/mweynants/mydatacubelinear.zarr", engine="zarr")
ds

<xarray.Dataset>
Dimensions:    (lat: 2445, lon: 2957, time: 336)
Coordinates:
  * lat        (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
    latitude   (lat) float64 ...
  * lon        (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
    longitude  (lon) float64 ...
  * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
Data variables:
    clc        (lat, lon) float32 ...
    d2m        (time, lat, lon) float32 ...
    skt        (time, lat, lon) float32 ...
    sp         (time, lat, lon) float32 ...
    t2m        (time, lat, lon) float32 ...
    tp         (time, lat, lon) float32 ...
    u10        (time, lat, lon) float32 ...
    v10        (time, lat, lon) float32 ...

## Add variable to existing cube {#minicube_add}
It is possible to add further variables to an existing zarr data cube. 

Here is an example in which we add a variable `dem` to the zarr group we've just created. The new variable doesn't have a time dimension, just longitude and latitude. 

In [6]:
dem = xr.open_dataset("/Net/Groups/BGI/scratch/fgans/uc3-mini-dataset/EU-DEM-CROPPED_NC/cropped_eu_dem_v11.nc")
dem

<xarray.Dataset>
Dimensions:  (lat: 9778, lon: 11824)
Coordinates:
  * lat      (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
  * lon      (lon) float64 20.82 20.82 20.82 20.82 ... 23.76 23.76 23.76 23.76
Data variables:
    Band1    (lat, lon) float32 ...
    crs      |S1 ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.0.2, released 2019/10/28
    history:      Tue Mar 30 10:42:42 2021: GDAL Create( EU-DEM-CROPPED_NC/cr...

In [22]:
dem.crs

<xarray.DataArray 'crs' ()>
array(b'', dtype='|S1')
Attributes:
    grid_mapping_name:            latitude_longitude
    long_name:                    CRS definition
    longitude_of_prime_meridian:  0.0
    semi_major_axis:              6378137.0
    inverse_flattening:           298.257223563
    spatial_ref:                  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...
    GeoTransform:                 20.82303858424455 0.0002486569144908497 0 3...

We need however first to make sure that the spatial grid matches that of our cube. 

`dem`'s CRS is EPSG:4326, like `clc`.

In [20]:
print(dem.dims)
print(ds.dims)
print(dem.dims["lat"] / ds.dims["lat"])
print(dem.dims["lon"] / ds.dims["lon"])

Frozen({'lat': 9778, 'lon': 11824})
Frozen({'lat': 2445, 'lon': 2957, 'time': 336})
3.9991820040899797
3.998647277646263


`dem` needs to be aggregated to match `ds` resolution. We convert it into dask arrays with a single block and aggregate along lon and lat using nearest neighbours (which is ot the best choice, but the fastest in this demo case).

In [15]:
dem_interp = dem.chunk().interp({'lon':target_lon, 'lat':target_lat},  method = 'nearest')

/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devn

In [16]:
dem_interp

<xarray.Dataset>
Dimensions:  (lat: 2445, lon: 2957)
Coordinates:
  * lon      (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
  * lat      (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
Data variables:
    Band1    (lat, lon) float32 dask.array<chunksize=(2445, 2957), meta=np.ndarray>
    crs      |S1 b''
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.0.2, released 2019/10/28
    history:      Tue Mar 30 10:42:42 2021: GDAL Create( EU-DEM-CROPPED_NC/cr...

We add the new variable to the existsing cube.

In [20]:
output_var = {'dem': dem_interp['Band1']}
ds2 = xr.Dataset(output_var).chunk({'lon':200, 'lat':200}).to_zarr("/Net/Groups/BGI/scratch/mweynants/mydatacubelinear.zarr/", mode="a")
ds2

/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/User/homes/mweynants/miniconda3_devnode/envs/myenv_python39/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/User/homes/mweynants/miniconda3_devn

In [21]:
ds = xr.open_dataset("/Net/Groups/BGI/scratch/mweynants/mydatacubelinear.zarr", engine="zarr")
ds

<xarray.Dataset>
Dimensions:    (lat: 2445, lon: 2957, time: 336)
Coordinates:
  * lat        (lat) float64 36.03 36.03 36.03 36.03 ... 38.46 38.46 38.46 38.46
    latitude   (lat) float64 ...
  * lon        (lon) float64 20.82 20.82 20.83 20.83 ... 23.76 23.76 23.76 23.76
    longitude  (lon) float64 ...
  * time       (time) datetime64[ns] 2020-08-07 ... 2020-08-20T23:00:00
Data variables:
    clc        (lat, lon) float32 ...
    d2m        (time, lat, lon) float32 ...
    dem        (lat, lon) float32 ...
    skt        (time, lat, lon) float32 ...
    sp         (time, lat, lon) float32 ...
    t2m        (time, lat, lon) float32 ...
    tp         (time, lat, lon) float32 ...
    u10        (time, lat, lon) float32 ...
    v10        (time, lat, lon) float32 ...

## Data Cube generation using xcube-gen{#xcube}

### Prerequisites

- every dataset is in NetCDF format, can consist of multiple files, but should be in a single folder
- datasets should already be in equirectangular lon-lat coordinate system
- make a config file in yaml format, an example config file would be:

### Prepare yaml file
Save configuration file as xcube/config.yaml. For example:

In [ ]:
output_size: [2000,2000]
output_region: [20, 36, 24, 39]
output_writer_name: 'zarr'

output_metadata:
  created_by: 'NOA'
  contact_email: 'yourmail@example.com'


- see [xcube-gen docs](https://xcube.readthedocs.io/en/latest/cli/xcube_gen.html) for more options
- make sure that all static variables have a time dimension (use `ds.expand_dims`)
- and add `time_bnds` attribute to them so they can be processed by the xcube default processor

## Run xcube gen
Cube the data with :


````
xcube gen -c xcube/config.yaml -o ./xcc.zarr ${PATH1}/*.nc ${PATH2}/cropped_CLC_2018_time.nc
````


## Rechunking{#rechunk}
If the number of chunks is too large, the xarray function to_zarr will fail to write the dask array to a zarr group.

### Rechunker{#rechunker}
Look into Python package [Rechunker](https://rechunker.readthedocs.io/en/latest/tutorial.html) to re-write Zarr with different chunks.

In [ ]:
%conda install rechunker

In [ ]:
from rechunker import rechunk
import xarray as xr
xr.set_options(display_style='text')
import zarr
import dask.array as dsa

ds = xr.tutorial.open_dataset("air_temperature")
# create initial chunk structure
ds = ds.chunk({'time': 100})
ds.air.encoding = {} # helps when writing to zarr
ds

In [ ]:
# clean up any existing temporary data
! rm -rf *.zarr 
# write to zarr
ds.to_zarr('air_temperature.zarr')
# open zarr group
source_group = zarr.open('air_temperature.zarr')
print(source_group.tree())


In [ ]:
# look at one array
source_array = source_group['air']
source_array.info

In [ ]:
# rechunk Group
target_chunks = {
    'air': {'time': 2920, 'lat': 25, 'lon': 1},
    'time': None, # don't rechunk this array
    'lon': None,
    'lat': None,
}
max_mem = '1MB'

target_store = 'group_rechunked.zarr'
# define temporate storage
temp_store = 'group_rechunked-tmp.zarr'

# need to remove the existing stores or it won't work
!rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
# prepare empty output zarr
array_plan = rechunk(source_group, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

In [ ]:
# execute plan
array_plan.execute()


In [ ]:
xr.open_zarr('group_rechunked.zarr')

### Rechunking in Julia{#jlrechunk}

Dask often fails to rechunk large cubes. We provide code in Julia for that purpose. See example in [datacube_rechunk.jl](./datacube_rechunk_example.jl).

### NCO ncap2{#ncrechunk}
Rechunk the netcdf with the [netCDF Operator (NCO)](http://nco.sourceforge.net/nco.html) [ncap2](http://nco.sourceforge.net/nco.html#ncap2-netCDF-Arithmetic-Processor). See [Documentation on Chunking](http://nco.sourceforge.net/nco.html#Chunking) for details on usage.

NCO can be installed with Anaconda (doesn't seem to be working for me on macadamia in myenv_python39:

In [ ]:
conda install -c conda-forge nco # Linux or MacOS with Anaconda

For example, a netCDF cube with fire data is re-chunked so that the chunk size of the time dimension is 1 (i.e. 1 time step by chunk), while x and y have 983 and 1253 respectively.

In [ ]:
ncap2 --cnk_dmn time,1 --cnk_dmn x,983 --cnk_dmn y,1253 ~/FireCube.nc  ~/FireCube_time1_x983_y1253.nc

In python, write an empty zarr file with the correct metadata:

In [ ]:
import xarray as xr
import zarr
import dask.array as dsa

zarr_path = "myzarr.zarr"
ds.chunk({'time': 1}).to_zarr(zarr_path, compute=False)

Then, write the data with xarray in pieces of 100 chunks:

In [ ]:
step = 100
for i in range (0, len(ds['time']), step):
      start, stop = (i, i+step)
      new_ds.isel(time=slice(start, stop)).to_zarr(zarr_path, 
region={"time": slice(start, stop)})
      print(f'Written time ({start}-{stop})')


## Create and deploy cube with panGeoForge recipes

see doc in https://pangeo-forge.readthedocs.io/

Install panGeoForge recipes python package

In [ ]:
conda install -c conda-forge pangeo-forge-recipes

TO DO: look into XarrayZarrRecipe
see how we can start from there
https://pangeo-forge.readthedocs.io/en/latest/pangeo_forge_recipes/recipe_user_guide/recipes.html

A Recipe is a Python object, which encapsulates a workflow for transforming data.
It takes a FilePattern (describing a collection of source files) and turns it into a single analysis-ready, cloud-optimized dataset.
The recipe can then be executed.

The idea is to develop recipes for specific data format of source and target.
Source needs to be defined clearly... Data format and file pattern.

So I will need to have recipes that get 
from source :
- gridded netCDF data?
to target:
- ovh store
- object store on some DIAS (once I get the info from GAEL)

In [ ]:
from pangeo_forge_recipes.recipes import XarrayZarrRecipe
XarrayZarrRecipe?

The [pangeo_forge_recipes.recipes.XarrayZarrRecipe](https://pangeo-forge.readthedocs.io/en/latest/pangeo_forge_recipes/api_reference.html#pangeo_forge_recipes.recipes.XarrayZarrRecipe) recipe class uses [Xarray](http://xarray.pydata.org/) to read the input files and [Zarr](https://zarr.readthedocs.io/) as the target dataset format. The inputs can be in [any file format Xarray can read](http://xarray.pydata.org/en/latest/user-guide/io.html), including NetCDF, OPeNDAP, GRIB, Zarr, and, via [rasterio](https://rasterio.readthedocs.io/), GeoTIFF and other geospatial raster formats. The target Zarr dataset can be written to any storage location supported by [filesystem-spec](https://filesystem-spec.readthedocs.io/); see [Storage](https://pangeo-forge.readthedocs.io/en/latest/pangeo_forge_recipes/recipe_user_guide/storage.html) for more details. The target Zarr dataset will conform to the [Xarray Zarr encoding conventions](http://xarray.pydata.org/en/latest/internals/zarr-encoding-spec.html).

In [ ]:
# 1. Define FilePattern

# 2. Define target storage (+ cache [and metadata] storage)
#  from pangeo_forge_recipes.storage import CacheFSSpecTarget, FSSpecTarget, MetadataTarget, StorageConfig

# define your fsspec filesystems for the target, cache, and metadata locations here

target = FSSpecTarget(fs=<fsspec-filesystem-for-target>, root_path="<path-for-target>")
cache = CacheFSSpecTarget(fs=<fsspec-filesystem-for-cache>, root_path="<path-for-cache>")
metadata = MetadataTarget(fs=<fsspec-filesystem-for-metadata>, root_path="<path-for-metadata>")

storage_config = StorageConfig(target, cache, metadata)

# 3. Call recipe
recipe = XarrayZarrRecipe(
    file_pattern = FilePattern,
    inputs_per_chunk = int,
    target_chunks = Dict[str,int],
    storge_config = storage_config
)

## Execution : different options
### A. manual execution (will probably be dropped)
# 1. Cache chunks
for input_key in recipe.iter_inputs():
    recipe.cache_input(input_key)
# 2. Prepare target : prepares zarr group on target storage with coords (no data vars yet)
recipe.prepare_target()
# 3. Write chunks to target
for chunk_key in recipe.iter_chunks():
    recipe.store_chunk(chunk_key)
#4. Finalise (cleanup and consolidation)
recipe.finalize_target()

### B. create python function
recipe_func = recipe.to_function()
recipe_func() # actually executes the recipe

### C. Dask Delayed
delayed = recipe.to_dask()
delayed.compute()
# can be executed by any of Dask's schedules, incl.cloud and HPC distributed schedulers.
